In [377]:
import requests
import pandas as pd
import numpy as np

url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Nombre de joueurs récupérés :", len(data["elements"]))
    # Exemple : afficher le nom du premier joueur
    print("Premier joueur :", data["elements"][0]["web_name"])
else:
    print("Erreur lors de la requête :", response.status_code)

Nombre de joueurs récupérés : 736
Premier joueur : Raya


In [378]:
df_teams = pd.DataFrame(data['teams'])[["id", "name", "short_name", "strength", "strength_overall_home", "strength_overall_away", "strength_attack_home", "strength_attack_away", "strength_defence_home", "strength_defence_away"]]

In [379]:
df_teams["avg_strenght"] = (df_teams["strength_overall_home"] + df_teams["strength_overall_away"]) / 2
df_teams.sort_values(by=["strength", "avg_strenght"] , inplace=True, ascending=False)

In [380]:
df_teams

,id,name,short_name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,avg_strenght
11,12,Liverpool,LIV,5,1335,1355,1290,1330,1380,1380,1345.0
0,1,Arsenal,ARS,4,1320,1325,1350,1350,1290,1300,1322.5
12,13,Man City,MCI,4,1275,1315,1250,1250,1300,1380,1295.0
6,7,Chelsea,CHE,4,1185,1245,1150,1190,1220,1300,1215.0
14,15,Newcastle,NEW,4,1185,1245,1130,1170,1240,1320,1215.0
1,2,Aston Villa,AVL,3,1125,1250,1110,1200,1140,1300,1187.5
15,16,Nott'm Forest,NFO,3,1165,1205,1150,1230,1180,1180,1185.0
3,4,Bournemouth,BOU,3,1150,1180,1100,1160,1200,1200,1165.0
5,6,Brighton,BHA,3,1150,1175,1090,1140,1210,1210,1162.5
4,5,Brentford,BRE,3,1120,1185,1080,1080,1160,1290,1152.5


In [381]:
df_positions = pd.DataFrame(data['element_types'])[["id", "singular_name_short"]]

In [382]:
df_positions.set_index('id', inplace=True)

In [383]:
df_positions

,singular_name_short
id,
1,GKP
2,DEF
3,MID
4,FWD


In [384]:
df_players = pd.DataFrame(data['elements'])[['id', 'first_name', 'web_name', 'team', 'element_type', 'now_cost', 'total_points',
'ict_index', 'selected_by_percent', 'selected_rank', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event','minutes', 'expected_goals_per_90', 'saves_per_90',
'expected_assists_per_90', 'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90', 'goals_conceded_per_90',
'defensive_contribution_per_90']]

In [385]:
# Joindre le nom de l'équipe sur la colonne 'team' (qui contient l'ID)
df_players = df_players.merge(df_teams[['id','name', 'short_name']], left_on='team', right_on='id', suffixes=('', '_team'))
# Joindre la position du joueur sur la colonne 'element_type' (ID)
df_players = df_players.merge(df_positions, left_on='element_type', right_on='id', suffixes=('', '_pos'))
# Ajout d'une colonne points total / prix
df_players.insert(loc=6, column="points_per_cost", value=df_players['total_points'] / df_players['now_cost'])
# Tri
df_players.sort_values(by=["points_per_cost", "total_points", "ict_index"] , inplace=True, ascending=False)

In [386]:
# Sélectionner les colonnes intéressantes pour plus de lisibilité
df_players = df_players[[
'id', 'first_name', 'web_name', 'short_name', 'name', 'singular_name_short', 'now_cost', 'total_points',
'ict_index', 'selected_by_percent', 'selected_rank', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event','minutes', 'expected_goals_per_90', 'saves_per_90',
'expected_assists_per_90', 'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90', 'goals_conceded_per_90',
'defensive_contribution_per_90'    
]]

# Renommer pour clarté
df_players = df_players.rename(columns={
    'name': 'team_name',
    'short_name': 'team_short',
    'singular_name_short': 'position',
    'now_cost': 'price'
})

In [387]:
df_players.set_index("id", inplace=True)

In [388]:
df_players

,first_name,web_name,team_short,team_name,position,price,total_points,ict_index,selected_by_percent,selected_rank,...,transfers_out,transfers_out_event,minutes,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90
id,,,,,,,,,,,,,,,,,,,,,
72,Marcos,Senesi,BOU,Bournemouth,DEF,46,25,12.5,5.0,82,...,60399,21026,270,0.05,0.0,0.03,0.08,0.95,1.33,14.33
226,Trevoh,Chalobah,CHE,Chelsea,DEF,51,27,16.1,6.4,65,...,79158,24536,248,0.19,0.0,0.09,0.28,0.71,0.36,9.07
260,Marc,Guéhi,CRY,Crystal Palace,DEF,46,23,11.6,15.9,28,...,192444,46657,270,0.05,0.0,0.01,0.06,1.22,0.33,6.67
7,Riccardo,Calafiori,ARS,Arsenal,DEF,57,28,17.7,11.2,36,...,127691,33162,224,0.57,0.0,0.01,0.58,0.63,0.40,4.82
8,Jurriën,J.Timber,ARS,Arsenal,DEF,56,26,23.7,8.9,47,...,132027,34096,172,0.47,0.0,0.05,0.52,0.71,0.52,8.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,Fábio,Fábio Silva,WOL,Wolves,FWD,49,0,0.0,0.4,273,...,29236,7530,0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
657,Leon,Chiwome,WOL,Wolves,FWD,45,0,0.0,0.0,604,...,2763,468,0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
658,Nathan,Fraser,WOL,Wolves,FWD,44,0,0.0,2.0,124,...,102040,19488,0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
